In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import torch
from sklearn.model_selection import train_test_split


In [2]:
data = pd.read_csv('train.csv')

X = data.drop('label',axis=1).values
y= data['label'].values


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

X_train = torch.tensor(X_train, dtype=torch.float32) / 255.0
X_test  = torch.tensor(X_test, dtype=torch.float32)  / 255.0

y_train = torch.tensor(y_train, dtype=torch.long)
y_test  = torch.tensor(y_test, dtype=torch.long)


input_size = 784
output_size = 10

First Model, Linear

In [3]:
import torch.nn as nn

model = nn.Linear(input_size,output_size)

# Training

learning_rate = 0.01
n_iters = 30


loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(n_iters):
    # forward
    y_pred = model(X_train)

    #loss
    l=loss(y_pred,y_train)

    #grad 
    l.backward()
    
    optimizer.step()

    optimizer.zero_grad()
    
    #steps
    if epoch %5 ==0 : 
        print(f'epoch {epoch +1}:, loss ={l.item():.4f} ')


with torch.no_grad():
    y_pred_test = model(X_test)
    predicted_labels = torch.argmax(y_pred_test, dim=1)

    accuracy = (predicted_labels == y_test).float().mean()
    print(f"Test accuracy = {accuracy.item():.4f}")


epoch 1:, loss =2.3032 
epoch 6:, loss =0.9041 
epoch 11:, loss =0.5791 
epoch 16:, loss =0.4697 
epoch 21:, loss =0.4171 
epoch 26:, loss =0.3841 
Test accuracy = 0.8939


# Drawing & Predicting

In [4]:
import numpy as np
import torch
import matplotlib
import matplotlib.pyplot as plt

matplotlib.use("TkAgg")

canvas = np.zeros((28, 28), dtype=np.uint8)

fig, ax = plt.subplots()
im = ax.imshow(canvas, cmap="gray", vmin=0, vmax=255)
ax.set_title("Dessine un chiffre puis ferme la fenêtre")

def on_move(event):
    if event.button == 1 and event.inaxes:
        x, y = int(event.xdata), int(event.ydata)
        if 0 <= x < 28 and 0 <= y < 28:
            canvas[y, x] = 255   # <-- valeur visible
            im.set_data(canvas)
            fig.canvas.draw_idle()

fig.canvas.mpl_connect("motion_notify_event", on_move)
plt.show()

# Préparation du tensor
img = torch.tensor(canvas, dtype=torch.float32).reshape(1, 784) / 255.0

with torch.no_grad():
    logits = model(img)
    probs = torch.softmax(logits, dim=1)
    pred = torch.argmax(probs, dim=1).item()

print("Prediction:", pred)


Prediction: 2


Pretty bad model, let's improve it : MLP

In [5]:
import torch.nn as nn

class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(784, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 10)
        )

    def forward(self, x):
        return self.network(x)

model2 = MLP()
# Training

learning_rate = 0.01
n_iters = 100


loss2= nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model2.parameters(), lr=learning_rate)

for epoch in range(n_iters):
    # forward
    y_pred = model2(X_train)

    #loss
    l=loss(y_pred,y_train)

    #grad 
    l.backward()
    
    optimizer.step()

    optimizer.zero_grad()
    
    #steps
    if epoch %5 ==0 : 
        print(f'epoch {epoch +1}:, loss ={l.item():.4f} ')


with torch.no_grad():
    y_pred_test = model2(X_test)
    predicted_labels = torch.argmax(y_pred_test, dim=1)

    accuracy = (predicted_labels == y_test).float().mean()
    print(f"Test accuracy = {accuracy.item():.4f}")


epoch 1:, loss =2.3092 
epoch 6:, loss =0.8330 
epoch 11:, loss =0.4591 
epoch 16:, loss =0.4050 
epoch 21:, loss =0.3208 
epoch 26:, loss =0.2690 
epoch 31:, loss =0.2308 
epoch 36:, loss =0.2003 
epoch 41:, loss =0.1743 
epoch 46:, loss =0.1522 
epoch 51:, loss =0.1341 
epoch 56:, loss =0.1187 
epoch 61:, loss =0.1060 
epoch 66:, loss =0.0949 
epoch 71:, loss =0.0851 
epoch 76:, loss =0.0766 
epoch 81:, loss =0.0691 
epoch 86:, loss =0.0625 
epoch 91:, loss =0.0565 
epoch 96:, loss =0.0511 
Test accuracy = 0.9614


In [6]:
X_train.shape

torch.Size([33600, 784])

In [ ]:
matplotlib.use("TkAgg")

canvas = np.zeros((28, 28), dtype=np.uint8)

fig, ax = plt.subplots()
im = ax.imshow(canvas, cmap="gray", vmin=0, vmax=255)
ax.set_title("Dessine un chiffre puis ferme la fenêtre")

def on_move(event):
    if event.button == 1 and event.inaxes:
        x, y = int(event.xdata), int(event.ydata)
        if 0 <= x < 28 and 0 <= y < 28:
            canvas[y, x] = 255   # <-- valeur visible
            im.set_data(canvas)
            fig.canvas.draw_idle()

fig.canvas.mpl_connect("motion_notify_event", on_move)
plt.show()

# Préparation du tensor
img = torch.tensor(canvas, dtype=torch.float32).reshape(1, 784) / 255.0

with torch.no_grad():
    logits = model2(img)
    probs = torch.softmax(logits, dim=1)
    pred = torch.argmax(probs, dim=1).item()

print("Prediction:", pred)


invalid command name "6284356416process_stream_events"
    while executing
"6284356416process_stream_events"
    ("after" script)


Prediction: 1


Next one : CNN 

# CNN

In [ ]:
X_train = X_train.view(-1, 1, 28, 28)
X_test  = X_test.view(-1, 1, 28, 28)
# Format attendu dans le bloc de convolution (33600 , 1 , 28 ,28)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader 

class CNN(nn.Module): 
    def __init__(self): 
        super().__init__() 
        self.network = nn.Sequential( #Conv 1 
            nn.Conv2d(1, 16, kernel_size=3, padding=1), 
            nn.ReLU(), 
            nn.MaxPool2d(2), 
        
            #Conv 2 
            nn.Conv2d(16, 32, kernel_size=3, padding=1), 
            nn.ReLU(), 
            nn.MaxPool2d(2),
            
            nn.Flatten(), 
            nn.Linear(32 * 7 * 7, 128), 
            nn.ReLU(), 
            nn.Linear(128, 10) ) 
        
    def forward(self, x): 
        return self.network(x) 
        
        
model3 = CNN() 
#Training 
batch_size = 64 
learning_rate = 0.01 
n_iters = 30 

train_dataset = TensorDataset(X_train, y_train) 
test_dataset = TensorDataset(X_test, y_test) 

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True) 
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False) 

criterion= nn.CrossEntropyLoss() 
optimizer = torch.optim.Adam(model3.parameters(), lr=learning_rate) 

for epoch in range(n_iters): 
    model3.train() 
    epoch_loss = 0.0 
    for X_batch, y_batch in train_loader: 
        # Forward 
        y_pred = model3(X_batch) 
        loss = criterion(y_pred, y_batch) 
        
        # Backward 
        optimizer.zero_grad() 
        loss.backward() 
        optimizer.step() 
        epoch_loss += loss.item() 
        
        if epoch % 5 == 0: 
            print(f"Epoch {epoch+1}, loss = {epoch_loss/len(train_loader):.4f}")
        
model3.eval() 
correct = 0 
total = 0 

with torch.no_grad(): 
    for X_batch, y_batch in test_loader: 
        outputs = model3(X_batch)
        _, predicted = torch.max(outputs, 1) 
        total += y_batch.size(0) 
        correct += (predicted == y_batch).sum().item()
        
accuracy = correct / total 
print(f"Test accuracy = {accuracy:.4f}")


In [30]:
matplotlib.use("TkAgg")

canvas = np.zeros((28, 28), dtype=np.uint8)

fig, ax = plt.subplots()
im = ax.imshow(canvas, cmap="gray", vmin=0, vmax=255)
ax.set_title("Dessine un chiffre puis ferme la fenêtre")

def on_move(event):
    if event.button == 1 and event.inaxes:
        x, y = int(event.xdata), int(event.ydata)
        if 0 <= x < 28 and 0 <= y < 28:
            canvas[y, x] = 255   # <-- valeur visible
            im.set_data(canvas)
            fig.canvas.draw_idle()

fig.canvas.mpl_connect("motion_notify_event", on_move)
plt.show()

# Préparation du tensor
img = torch.tensor(canvas, dtype=torch.float32).reshape(1, 784).view(-1, 1, 28, 28) / 255.0

with torch.no_grad():
    logits = model3(img)
    probs = torch.softmax(logits, dim=1)
    pred = torch.argmax(probs, dim=1).item()

print("Prediction:", pred)


Prediction: 6
